## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-06-20-47-45 +0000,nypost,Richest US real estate baron disowns son in sa...,https://nypost.com/2025/10/06/business/richest...
1,2025-10-06-20-44-52 +0000,48hills,"Two decades in, you still can’t break Broke-As...",https://48hills.org/2025/10/two-decades-in-you...
2,2025-10-06-20-44-48 +0000,nyt,"Trump Aimed Shutdown Cuts at Democrats, but G....",https://www.nytimes.com/2025/10/06/us/politics...
3,2025-10-06-20-44-43 +0000,nyt,"Man Breaks Into Washington State Capitol, Topp...",https://www.nytimes.com/2025/10/06/us/washingt...
4,2025-10-06-20-43-55 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/06/us/nat...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,45
130,new,19
111,judge,15
232,gaza,14
64,government,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
27,2025-10-06-20-04-32 +0000,nypost,Trump appealing ruling against Portland Nation...,https://nypost.com/2025/10/06/us-news/trump-ap...,103
34,2025-10-06-19-54-25 +0000,nypost,Judge rejects Trump admin’s bid to delay Kilma...,https://nypost.com/2025/10/06/us-news/judge-re...,101
68,2025-10-06-18-34-32 +0000,nypost,Trump pushes for his Gaza peace deal to be put...,https://nypost.com/2025/10/06/us-news/trump-pu...,98
214,2025-10-06-08-50-56 +0000,wapo,Judge deals Trump new setback in plans to depl...,https://www.washingtonpost.com/nation/2025/10/...,97
87,2025-10-06-17-52-24 +0000,nypost,Retired IDF general who saved his family on Oc...,https://nypost.com/2025/10/06/world-news/retir...,93


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
27,103,2025-10-06-20-04-32 +0000,nypost,Trump appealing ruling against Portland Nation...,https://nypost.com/2025/10/06/us-news/trump-ap...
44,81,2025-10-06-19-34-39 +0000,cbc,Bari Weiss named editor-in-chief of CBS News u...,https://www.cbc.ca/news/entertainment/bari-wei...
91,54,2025-10-06-17-44-15 +0000,wapo,Prime minister of France resigns hours after n...,https://www.washingtonpost.com/world/2025/10/0...
259,44,2025-10-05-23-07-07 +0000,bbc,Gisèle Pelicot to face one of her rapists in c...,https://www.bbc.com/news/articles/cdx2d9lynk1o...
237,42,2025-10-06-02-49-22 +0000,nypost,Greta Thunberg to be deported from Israel afte...,https://nypost.com/2025/10/05/world-news/greta...
145,35,2025-10-06-14-58-23 +0000,nypost,"Give President Trump Nobel Peace Prize, Israel...",https://nypost.com/2025/10/06/world-news/give-...
117,34,2025-10-06-16-21-57 +0000,nypost,Eric Adams jets off to Albania after dropping ...,https://nypost.com/2025/10/06/us-news/eric-ada...
52,31,2025-10-06-19-11-11 +0000,bbc,US Supreme Court rejects Ghislaine Maxwell app...,https://www.bbc.com/news/articles/cn83e6q7lq4o...
111,30,2025-10-06-16-33-42 +0000,nypost,Aston Martin shares skid 10% as luxury carmake...,https://nypost.com/2025/10/06/business/aston-m...
106,29,2025-10-06-16-48-36 +0000,bbc,Sudan militia leader convicted of war crimes d...,https://www.bbc.com/news/articles/cly1egd5d0vo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
